В плане тестирования несколько пунктов относятся к маршрутам, посмотрим на них ещё раз:

1. Главная страница доступна анонимному пользователю. *[Done!]*

2. Страница отдельной новости доступна анонимному пользователю.

3. Страницы удаления и редактирования комментария доступны автору комментария.

4. При попытке перейти на страницу редактирования или удаления комментария анонимный пользователь перенаправляется на страницу авторизации.

5. Авторизованный пользователь не может зайти на страницу редактирования или удаления чужих комментариев (возвращается ошибка 404).

6. Страницы регистрации пользователей, входа в учётную запись и выхода из неё доступны анонимным пользователям.

В вашем файле *news/tests/test_routes.py* сейчас должен быть код теста, проверяющего доступность главной страницы проекта:

In [ ]:
# news/tests/test_routes.py
from http import HTTPStatus

from django.test import TestCase
from django.urls import reverse


class TestRoutes(TestCase):

    def test_home_page(self):
        url = reverse('news:home')
        response = self.client.get(url)
        self.assertEqual(response.status_code, HTTPStatus.OK) 

***
## Доступность отдельной страницы новости

Чтобы протестировать доступность страницы отдельной новости, для начала надо создать новость — ведь БД, на которой проводится тестирование, перед запуском тестов не содержит никаких объектов.

***
## **Создаём объект для теста**

Добавляем фикстуру с созданием первой новости:


In [ ]:
# news/tests/test_routes.py
...  

# Импортируем класс модели новостей.
from news.models import News


class TestRoutes(TestCase):

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')

    ... 

Сейчас неважно, какой именно заголовок и текст будут у новости. При тестировании маршрутов содержимое свойств объекта проверяться не будет; значения этих строк не нужно сохранять в константы: в коде тестов эти строки больше нигде не пригодятся.

***
## **Получаем адрес страницы для запроса**

Строки кода, описывающие URL к странице новости, выглядят так:


In [ ]:
# news/urls.py
...

app_name = 'news'

urlpatterns = [
    ...,
    path('news/<int:pk>/', views.NewsDetailView.as_view(), name='detail'),
    ...,
] 

Адресом страницы с новостью, созданной в фикстуре, будет */news/1/*: нам точно известно, что это первый объект, который создан в пустой таблице БД. 

Единица в адресе — это id новости и, одновременно, её первичный ключ (Primary Key, pk). Если в описании модели явно не указано, какое поле служит первичным ключом (а в модели `News` это не указано), то Django автоматически добавляет в модель поле с первичным ключом. 

Настройки этого поля для конкретного приложения задаются через AppConfig.`default_auto_field`, а глобально, для всего проекта — в *settings.py* через `DEFAULT_AUTO_FIELD`. Тип этого поля по умолчанию — `BigAutoField` (целое число от 1 до 9223372036854775807).

Несмотря на то, что адрес страницы известен — не стоит «хардкодить», писать этот адрес в явном виде, строкой `'/news/1/'`. Если структура адресов поменяется — придётся менять и эту строку с адресом везде, где она используется. 

Есть и другая причина: может быть написана ещё одна фикстура, которая, например, создаст объект класса `News` для всего тестирующего модуля. И тогда адрес страницы для объекта, созданного в фикстуре класса, будет другой.

Правильным решением будет обратиться к адресу страницы через функцию `reverse()`. В неё надо передать 

* `namespace` и `name` страницы — увидеть их можно в файле *urls.py* приложения; в нашем случае это `'news:detail'`

* pk (Primary Key) записи; в нашем случае pk совпадает с id.

Передать необходимые значения в функцию `reverse()` можно через позиционные или именованные аргументы:

In [ ]:
url = reverse('news:detail', args=(self.news.pk,))
# Или      
url = reverse('news:detail', kwargs={'pk': self.news.pk})

В каждом из приведённых вариантов происходит одно и то же — переменной `url` присваивается строка `'/news/1/'`. То же самое можно сделать, указывая id объекта, а не pk. При этом нужно помнить, что вообще-то первичный ключ не всегда равен id, и в каких-то проектах он может быть другим.


In [ ]:
url = reverse('news:detail', args=(self.news.id,))
# Или
url = reverse('news:detail', kwargs={'pk': self.news.id}) 

Далее в тестах мы будем использовать вариант `reverse('news:detail', args=(self.news.id,))`, но вы можете использовать любой предложенный, кроме явного «хардкода» — строки `/news/1/`. 

Полный код теста будет выглядеть так:


In [ ]:
# news/tests/test_routes.py
from http import HTTPStatus

from django.test import TestCase
from django.urls import reverse

from news.models import News


class TestRoutes(TestCase):

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')

    def test_home_page(self):
        url = reverse('news:home')
        response = self.client.get(url)
        self.assertEqual(response.status_code, HTTPStatus.OK)

    def test_detail_page(self):
        url = reverse('news:detail', args=(self.news.id,))
        response = self.client.get(url)
        self.assertEqual(response.status_code, HTTPStatus.OK) 

Тесты `test_home_page()` и `test_detail_page()` работают нормально, но их можно улучшить. В этих тестах повторяется один и тот же код: одинаково вызывается функция `reverse()`, одинаково отправляется GET-запрос к проверяемой странице, одинаково проверяется статус ответа. Разница между этими тестами — только в адресах страниц. 

Вспомним о принципе DRY и объединим эти два теста в один: соберём в кортеж аргументы для функции `reverse()` и применим метод `subTest()`:


In [ ]:
# news/tests/test_routes.py
from http import HTTPStatus

from django.test import TestCase
from django.urls import reverse

from news.models import News


class TestRoutes(TestCase):

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')

    def test_pages_availability(self):
        # Создаём набор тестовых данных - кортеж кортежей.
        # Каждый вложенный кортеж содержит два элемента:
        # имя пути и позиционные аргументы для функции reverse().
        urls = (
            # Путь для главной страницы не принимает
            # никаких позиционных аргументов, 
            # поэтому вторым параметром ставим None.
            ('news:home', None),
            # Путь для страницы новости 
            # принимает в качестве позиционного аргумента
            # id записи; передаём его в кортеже.
            ('news:detail', (self.news.id,))
        )
        # Итерируемся по внешнему кортежу 
        # и распаковываем содержимое вложенных кортежей:
        for name, args in urls:
            with self.subTest(name=name):
                # Передаём имя и позиционный аргумент в reverse()
                # и получаем адрес страницы для GET-запроса:
                url = reverse(name, args=args)
                response = self.client.get(url)
                self.assertEqual(response.status_code, HTTPStatus.OK) 

Выглядит как излишнее усложнение: ведь нужно проверить всего две страницы. Но в плане тестирования есть пункт 6: «Страницы регистрации пользователей, входа в учётную запись и выхода из неё доступны анонимным пользователям». 

Страницы логина и регистрации пользователей можно проверить в том же тесте `test_pages_availability()`, нужно только дополнить кортеж с параметрами.

Имена, по которым доступны нужные страницы, берём в головном файле *yanews/urls*.py.


In [ ]:
# news/tests/test_routes.py
from http import HTTPStatus

from django.test import TestCase
from django.urls import reverse

from news.models import News


class TestRoutes(TestCase):

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')

    def test_pages_availability(self):
        urls = (
            ('news:home', None),
            ('news:detail', (self.news.id,)),
            ('users:login', None),
            ('users:signup', None),
        )
        for name, args in urls:
            with self.subTest(name=name):
                url = reverse(name, args=args)
                response = self.client.get(url)
                self.assertEqual(response.status_code, HTTPStatus.OK) 

Четыре адреса проверяются одним тестом! Становится очевидно, насколько `subTest()` облегчает работу и почему не стоит писать отдельные тесты для подобных случаев.

Второй и шестой пункты плана тестирования готовы! 

***
## **Проверка страниц редактирования и удаления комментария**

Теперь проверим, что страницы редактирования и удаления комментария доступны автору комментария, но недоступны другому пользователю. 

Для этого в тестах понадобятся:

* Пользователь — автор комментария (author).

* Другой пользователь, читатель (reader).

* Собственно, сам комментарий.

Дополните файл news/tests/test_routes.py следующими строками:


In [ ]:

# news/tests/test_routes.py
from http import HTTPStatus

# Импортируем функцию для определения модели пользователя.
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse

# Импортируем класс комментария.
from news.models import Comment, News

# Получаем модель пользователя.
User = get_user_model()


class TestRoutes(TestCase):

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')
        # Создаём двух пользователей с разными именами:
        cls.author = User.objects.create(username='Лев Толстой')
        cls.reader = User.objects.create(username='Читатель простой')
        # От имени одного пользователя создаём комментарий к новости:
        cls.comment = Comment.objects.create(
            news=cls.news,
            author=cls.author,
            text='Текст комментария'
        ) 


Для программного создания пользователей принято использовать [метод Django ORM](https://docs.djangoproject.com/en/5.1/topics/auth/default/#creating-users) `create_user()`, который правильным образом устанавливает пароль для пользователя. Но в тестах мы пароли не используем — и можем обойтись обычным `create()`.

Теперь надо проверить, что:

1. Автор может зайти на страницу редактирования своего комментария.

2. Автор может зайти на страницу удаления своего комментария.

3. Читатель не может зайти на страницу редактирования чужого комментария.

4. Читатель не может зайти на страницу удаления чужого комментария.

Эти проверки можно записать в виде `subTest()` на четыре строки, но этот путь мы уже проходили. Напишем код иначе: через вложенные циклы.

Соберём исходные данные в кортеж кортежей; вложенные кортежи будут содержать объект пользователя и ответ, который ожидается при обращении пользователя к проверяемым страницам:


In [ ]:

# При обращении к страницам редактирования и удаления комментария
users_statuses = (
    (self.author, HTTPStatus.OK),  # автор комментария должен получить ответ OK,
    (self.reader, HTTPStatus.NOT_FOUND),  # читатель должен получить ответ NOT_FOUND.
) 


В тесте применим вложенный цикл:


In [ ]:

# news/tests/test_routes.py
...
    def test_availability_for_comment_edit_and_delete(self):
        users_statuses = (
            (self.author, HTTPStatus.OK),
            (self.reader, HTTPStatus.NOT_FOUND),
        )
        for user, status in users_statuses:
            # Логиним пользователя в клиенте:
            self.client.force_login(user)
            # Для каждой пары "пользователь - ожидаемый ответ"
            # перебираем имена тестируемых страниц:
            for name in ('news:edit', 'news:delete'):  
                with self.subTest(user=user, name=name):        
                    url = reverse(name, args=(self.comment.id,))
                    response = self.client.get(url)
                    self.assertEqual(response.status_code, status) 


Перебираем в цикле кортеж `users_statuses`; на каждой итерации

* логиним каждого пользователя в клиенте: передаём переменную `user` в метод `force_login()`;

* во вложенном цикле перебираем имена страниц редактирования и удаления;

* в контекстном менеджере `subTest()` по знакомой схеме формируем адрес, выполняем запрос и проверяем ответ.

На первый взгляд конструкция может показаться сложной, но такой код получается гораздо короче и гибче, чем отдельные тесты для каждого пользователя и каждой страницы.

Ещё два пункта плана выполнены!

***
## **Проверка редиректов**

Переадресация — это последнее, что остаётся проверить в маршрутах. При попытке зайти на страницу редактирования или удаления чужого комментария анонимный пользователь должен быть переадресован на страницу логина.

При тестировании необходимо помнить, что при редиректе на страницу логина Django передаёт в URL параметр `next` — адрес страницы, куда пользователь будет перенаправлен после того, как авторизуется.

Если на страницу */user-only/*, к которой разрешён доступ только авторизованным пользователям, попытается зайти анонимный посетитель — этот посетитель будет перенаправлен на страницу логина с параметром *next=/user-only/*: 


В `django.test` есть специальный метод `assertRedirects`, он проверяет предположение о том, что с определённого адреса происходит редирект на другую страницу. 

Этот метод позволяет протестировать 

* статус ответа запрошенной страницы (по умолчанию — 302);

* статус ответа страницы, на которую ожидается редирект (по умолчанию — 200);

* адрес страницы, куда привела цепочка редиректов.

При тестировании редиректа нет смысла проверять каждый из этих пунктов по отдельности; применяйте метод `assertRedirects`.

Для двух страниц нужно провести одинаковые тесты; вновь применим `subTest()`:


In [ ]:
# news/tests/test_routes.py
...
    def test_redirect_for_anonymous_client(self):
        # Сохраняем адрес страницы логина:
        login_url = reverse('users:login')
        # В цикле перебираем имена страниц, с которых ожидаем редирект:
        for name in ('news:edit', 'news:delete'):
            with self.subTest(name=name):
                # Получаем адрес страницы редактирования или удаления комментария:
                url = reverse(name, args=(self.comment.id,))
                # Получаем ожидаемый адрес страницы логина, 
                # на который будет перенаправлен пользователь.
                # Учитываем, что в адресе будет параметр next, в котором передаётся
                # адрес страницы, с которой пользователь был переадресован.
                redirect_url = f'{login_url}?next={url}'
                response = self.client.get(url)
                # Проверяем, что редирект приведёт именно на указанную ссылку.
                self.assertRedirects(response, redirect_url) 

В начале теста сохраняем ссылку на страницу логина в переменную `login_url`. Затем в цикле перебираем имена страниц; на каждой итерации цикла получаем адрес запрошенной страницы и адрес редиректа. 

Анонимным клиентом «стучимся» по адресу и проверяем, что произошёл редирект. 

Добавьте новый тест в test_routes.py, запустите тесты.

Если всё ОК — значит, план по тестированию маршрутов выполнен!

Если какой-то из тестов провален — внимательно прочтите отчёт о тестировании, проверьте свой код и сверьте его с примерами из урока. Всё заработает!